In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import fitsio
from astropy.io import fits
import pandas as pd 
import sys
from quickbin import bin2d
from pyarrow import parquet
import pyarrow

In [ ]:
def centile_clip(image, centiles=(1, 99.5)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
eclipse = 'e23456'

image = fits.open(f"/home/bekah/gPhoton2/test_data/{eclipse}/{eclipse}-nd-ffull-b00-image-r.fits")
movie = fits.open(f"/home/bekah/gPhoton2/test_data/{eclipse}/{eclipse}-nd-f0120-b00-movie-r.fits") 

In [ ]:

def prep_image_inputs(photonfile, edge_threshold):
    event_table, exposure_array = load_image_tables(photonfile)
    foc, wcs = generate_wcs_components(event_table)
    with warnings.catch_warnings():
        # don't bother us about divide-by-zero errors
        warnings.simplefilter("ignore")
        weights = 1.0 / event_table["response"].to_numpy()
    mask_ix = np.where(event_table["mask"].to_numpy())
    edge_ix = np.where(event_table["detrad"].to_numpy() > edge_threshold)
    t = event_table["t"].to_numpy()
    map_ix_dict = generate_indexed_values(edge_ix, foc, mask_ix, t, weights)
    total_trange = (t.min(), t.max())
    return exposure_array, map_ix_dict, total_trange, wcs


def generate_indexed_values(edge_ix, foc, mask_ix, t, weights):
    indexed = NestingDict()
    for value, value_name in zip((t, foc, weights), ("t", "foc", "weights")):
        for map_ix, map_name in zip(
            (edge_ix, mask_ix, slice(None)), ("edge", "flag", "cnt")
        ):
            indexed[map_name][value_name] = value[map_ix]
    return indexed

def load_image_tables(
    photonfile: Pathlike,
) -> tuple[pyarrow.Table, np.ndarray]:
    """
    read a photonlist file produced by `photonpipe` from a raw6 telemetry file;
    return event and exposure tables appropriate for making images / movies
    and performing photometry
    """
    relevant_columns = ["ra", "dec", "response", "flags", "mask", "t", "detrad"]
    event_table = parquet.read_table(photonfile, columns=relevant_columns)
    # retain time and flag for every event for exposure time computations
    exposure_array = parquet_to_ndarray(event_table, ["t", "flags"])
    # but otherwise only deal with data actually on the 800x800 detector grid
    event_table = select_on_detector(event_table)
    return event_table, exposure_array

def parquet_to_ndarray(table, columns=None):
    if columns is None:
        columns = table.column_names
    return np.array([table[column].to_numpy() for column in columns]).T

def select_on_detector(
    event_table: pyarrow.Table, threshold: int = 400
) -> pyarrow.Table:
    """
    select events "on" the detector
    :param event_table: pyarrow Table that contains a detector radius column
    :param threshold: how many pixels away from center still counts as "on"?
    :return: Table consisting of rows of event_table "on" detector
    """
    detrad = event_table["detrad"].to_numpy()
    return event_table.take(
        # TODO: is isfinite() necessary?
        np.where(np.isfinite(detrad) & (detrad < threshold))[0]
    )

def generate_wcs_components(event_table):
    wcs = make_bounding_wcs(parquet_to_ndarray(event_table, ["ra", "dec"]))
    # This is a bottleneck, so only do it once.
    # TODO: do we actually want these 1-indexed?
    # TODO: are we supposed to have SIP correction? we don't appear to.
    foc = wcs.sip_pix2foc(
        wcs.wcs_world2pix(parquet_to_ndarray(event_table, ["ra", "dec"]), 1), 1
    )
    return foc, wcs

def make_bounding_wcs(
    radec: np.ndarray,
    pixsz: float = c.DEGPERPIXEL,
    proj: Sequence[str] = ("RA---TAN", "DEC--TAN")
) -> astropy.wcs.WCS:
    """
    makes a WCS solution for a given range of ra/dec values
    by default, assumes gnomonically-projected ra/dec values; scales ra bounds
    to approximate distortion in pixel size
    radec: n x 2 array with ra in first column and dec in second
    pixsz: size of returned WCS's pixels in square degrees;
    defaults to degree-per-pixel scale set in gPhoton.constants.DEGPERPIXEL
    """
    real_ra = radec[:, 0][np.isfinite(radec[:, 0])]
    real_dec = radec[:, 1][np.isfinite(radec[:, 1])]
    ra_range = real_ra.min(), real_ra.max()
    dec_range = real_dec.min(), real_dec.max()
    # handle viewports in which ra wraps around 360
    if ra_range[1] - ra_range[0] > 350:
        real_ra[real_ra > 180] -= 360
        ra_range = real_ra.min(), real_ra.max()
    # WCS center pixel in sky coordinates
    ra0, dec0 = (np.mean(ra_range), np.mean(dec_range))
    ra0 = ra0 if ra0 > 0 else ra0 + 360
    # scale ra-axis pixel size using cos(declination) to approximate
    # ra-direction distortion introduced by gnomonic projection
    ra_offset = (ra_range[1] - ra_range[0]) * math.cos(math.radians(dec0))
    imsz = (
        int(np.ceil((dec_range[1] - dec_range[0]) / pixsz)),
        int(np.ceil(ra_offset / pixsz)),
    )
    return make_wcs((ra0, dec0), imsz=imsz, pixsz=pixsz, proj=proj)


In [ ]:
def select_on_detector(
    event_table: pyarrow.Table, threshold: int = 400
) -> pyarrow.Table:
    """
    select events "on" the detector
    :param event_table: pyarrow Table that contains a detector radius column
    :param threshold: how many pixels away from center still counts as "on"?
    :return: Table consisting of rows of event_table "on" detector
    """
    detrad = event_table["detrad"].to_numpy()
    return event_table.take(
        # TODO: is isfinite() necessary?
        np.where(np.isfinite(detrad) & (detrad < threshold))[0]
    )


In [ ]:
edge_threshold = 350 
relevant_columns = ["ra", "dec", "response", "flags", "mask", "t", "detrad"]

photonfile = "/home/bekah/gPhoton2/test_data/e44405/e44405-nd-b00.parquet"
event_table = parquet.read_table(photonfile, columns=relevant_columns)
event_table = select_on_detector(event_table, edge_threshold)


In [ ]:
def calculate_flag_mask(flags):
    flag_column_mapping = {
        7: 0,
        12: 1,
        120: 2}
    mask = 0
    for flag, bit_position in flag_column_mapping.items():
        if flags & flag:
            mask |= (1 << bit_position)
    return mask

In [ ]:
mask_part = np.where(event_table["mask"].to_numpy(), (1 << 3), 0)
flag_part = event_table["flags"].to_numpy()

flag_mask = np.vectorize(calculate_flag_mask)(flag_part)
bitmap = mask_part | flag_mask



In [ ]:
mask_ix = np.where(event_table["mask"].to_numpy())
flag_ix = np.where(event_table["flags"].to_numpy() != 0)


In [ ]:
np.size(mask_ix)

In [ ]:
np.size(flag_ix)

In [ ]:
flag_to_bit = {7: 0, 12: 1, 120: 2}

mask_array = event_table["mask"].to_numpy()
flags_array = event_table["flags"].to_numpy()

valid_flags = np.array(list(flag_to_bit.keys()))
mask_ix = np.where((mask_array != 0) | (np.isin(flags_array, valid_flags)))[0]

def flags_to_bitmap(flag, is_masked):
    bitmap = 0
    for value, bit in flag_to_bit.items():
        if flag == value:
            bitmap |= (1 << bit)  
    if is_masked and bitmap == 0:
        bitmap = 8
    return bitmap

bitmap = np.array([
    flags_to_bitmap(flags_array[i], mask_array[i] != 0) for i in mask_ix
])

In [ ]:
# this is the one 

flag_to_bit = {7: 0, 12: 1, 120: 2}
mask_bit = 3

mask_array = event_table["mask"].to_numpy().astype(np.uint8)
flags_array = event_table["flags"].to_numpy().astype(np.uint8)

bitmap = np.zeros_like(flags_array, dtype=np.uint8)

for flag, bit in flag_to_bit.items():
    bitmap |= ((flags_array & flag) == flag).astype(np.uint8) << bit

bitmap |= (mask_array != 0).astype(np.uint8) << mask_bit



In [ ]:
unique = list(set(bitmap))
unique

In [ ]:
plt.hist(event_table['flags'], bins=15)

In [1]:
# wth fast histogram 
import fast_histogram as fh
import matplotlib.pyplot as plt
import numpy as np
import fitsio
from astropy.io import fits
import pandas as pd 

